### 1-1 누락 데이터 확인

In [9]:
import seaborn as sns

df = sns.load_dataset('titanic')

In [10]:
# info() 메소드
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


info() 메소드를 통해 확인한 결과,  
타이타닉 데이터셋의 총 891 개의 행 중, non-null (유효한 값)의 개수를 파악 할 수 있다.  

In [13]:
# value_counts() 메소드
print(df['deck'].value_counts(dropna=False))

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64


value_counts(dropna=False) 메소드를 통해 확인한 결과,  
nan 값이 688개 존재하는것을 알 수 있다.

In [23]:
# isnull() 메소드와 notnull() 메소드
print(df.head().isnull())
print('\n############################################################\n')
print(df.head().notnull())

# isnull() 메소드와 notnull() 메소드를 통한 누락 데이터 개수 확인
print(df.head().isnull().sum(axis=0))
print('\n############################################################\n')
print(df.head().notnull().sum(axis=0)) # axis=0 일 경우 각 행의 합을, axis=1 일 경우 하나의 행의 열의 합을 구한다.

# 반환값은 진위값으로 구성된 데이터프레임
print(type(df.head().isnull()))

survived  pclass    sex    age  sibsp  parch   fare  embarked  class  \
0     False   False  False  False  False  False  False     False  False   
1     False   False  False  False  False  False  False     False  False   
2     False   False  False  False  False  False  False     False  False   
3     False   False  False  False  False  False  False     False  False   
4     False   False  False  False  False  False  False     False  False   

     who  adult_male   deck  embark_town  alive  alone  
0  False       False   True        False  False  False  
1  False       False  False        False  False  False  
2  False       False   True        False  False  False  
3  False       False  False        False  False  False  
4  False       False   True        False  False  False  

############################################################

   survived  pclass   sex   age  sibsp  parch  fare  embarked  class   who  \
0      True    True  True  True   True   True  True      True   True 

isnull() 메소드는 데이터프레임의 각 원소에 대해 원소값이 누락 데이터일 경우 True 를 반환하고  
notnull() 메소드는 데이터프레임의 각 원소에 대해 원소값이 유효 데이터일 경우 True 를 반환한다.  
  
df.isnull() 의  반환값은 진위값으로 구성된 새로운 데이터프레임이다.  
해당 데이터프레임의 해당 열의 합을 통해 누락 데이터의 개수를 구할 수 있다.


### 1-2 누락 데이터 제거

In [24]:
import seaborn as sns
df = sns.load_dataset('titanic')

In [35]:
# isnull() 을 통해 각 원소가 NaN 인지에 대한 진위값 데이터프레임 생성
missing_df = df.isnull()

# for 반복문을 통한 각 열 NaN 개수 계산
for col_name in missing_df.columns:
    missing_count = missing_df[col_name].value_counts() # value_counts() 는 시리즈에 대해 고유한 값의 개수를 반환한다.
    
    try:        # missing_count 에 True 인덱스가 존재한다면, 그 개수를 출력
        print(col_name, ': ', missing_count[True])  
    except:     # missing_count 에 True 인덱스가 존재하지 않는다면, 0 을 출력
        print(col_name, ': ', 0)

survived :  0
pclass :  0
sex :  0
age :  177
sibsp :  0
parch :  0
fare :  0
embarked :  2
class :  0
who :  0
adult_male :  0
deck :  688
embark_town :  2
alive :  0
alone :  0


In [36]:
# 누락 데이터 포함 열 제거
df_thresh = df.dropna(axis=1, thresh=500)
print(df_thresh.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')


실제 머신러닝 분석을 할때도 deck 열과 같이 누락데이터가 많은 열은 제외하는 것이 좋다.  
위의 코드에서는 df 의 dropna() 메소드를 각 열에 대해 수행하는데,  
thresh 옵션을 통해 500 개 이상의 널값을 포함하는 열을 삭제한다.

In [41]:
# 누락 데이터 포함 행 제거
df_age = df.dropna(subset=['age'], how='any', axis=0)
print(len(df_age), df['age'].isnull().sum(),len(df))

714 177 891


age 열이 분석에 중요한 역할을 한다면, 해당 열에 널값이 있는 행을 제거하는것이 좋다.  
dropna() 메소드를 사용하고 모든 행에 대해 (axis=0), age 열에 대해 (subset=['age])  
Null 값이 하나라도 존재한다면 삭제한다 (how='any')

### 1-3 누락 데이터 치환

In [42]:
import seaborn as sns
df = sns.load_dataset('titanic')

In [57]:
# 특정 열의 평균값 계산
mean_age = df['age'].mean(axis=0)   # 산술연산이 가능한 값에 대해 평균값 계산

# 특정 열의 NaN값 치환
new_age_series = df['age'].fillna(mean_age)
df['age'] = new_age_series
print(df['age'].isnull().sum(), len(new_age_series) - new_age_series.isnull().value_counts()[False])

177 0


분석을 함에 있어 누락된 행/열을 전체 삭제하는 것 보다는 의미있는 데이터로 치환하는 것이  
더욱 학습 결과를 좋게 만드는 경우가 많다.  
  
치환할 수 있는 데이터는 평균값, 최빈값 등의 분포/특성을 잘 나타낼 수 있는 데이터를 활용한다.  
fillna() 메소드를 통해 해당 열의 NaN 값을 특정 값으로 채워 넣을 수 있다.

In [65]:
# 특정 열의 최빈값 계산
most_embark_town = df['embark_town'].value_counts(dropna=True).idxmax()

# 특정 열의 NaN 값 치환
df['embark_town'].fillna(most_embark_town, inplace=True)

print(df['embark_town'].isnull().sum())

0


최빈값을 찾기 위해, df['embark_town'] 열에 대해 각 고유 원소의 개수를 세고,  
그중 가장 큰 값을 가지는 원소를 idxmax() 메소드를 통해 추출한다.

만약, nan 과 같은 데이터로 누락 데이터가 표현 되지 않는 경우 (?, -, ...)  
누락 데이터를 찾아 nan 으로 바꾸어주는 것이 좋다.  
```ex) df.replace('?', np.nan, inplace=True)```

In [ ]:
# 특정 열의 nan 값을 이전 행의 값으로 치환
df['embark_town'].fillna(method='ffill') # front fill

# 특정 열의 nan 값을 다음 행의 값으로 치환
df['embark_town'].fillna(method='bfill') # back fill

### 2-1 중복 데이터 확인

In [67]:
import pandas as pd

df = pd.DataFrame({
    'c1': ['a','a','b','a','b'],
    'c2': [1, 1, 1, 2, 2],
    'c3': [1, 1, 2, 2, 2]
})

In [72]:
# duplicated() 메소드
df_dup = df.duplicated()
print(df)
print('\n')
print(df_dup)
print('\n')
print(type(df_dup))

c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


0    False
1     True
2    False
3    False
4    False
dtype: bool


<class 'pandas.core.series.Series'>


데이터프레임에서 하나의 행은 분석 대상이 갖고 있는 모든 속성(변수)에 대한 관측값(레코드)이다.  
하나의 데이터셋에서 동일한 관측값이 2개 이상 중복되는 경우 (행이 중복되는 경우)  
분석 결과를 왜곡하기 떄문에 중복 데이터를 찾아 삭제해야한다.  
  
df.duplicated() 메소드를 통해 전에 나온 행과 비교해 중복행일 경우 True 를 , 처음 행일 경우 False 를 가진 시리즈를 반환한다.



In [73]:
# 특정 열에 대해 중복되는 행 찾기
col_dup = df['c2'].duplicated()
print(col_dup)

0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool


### 2-2 중복 데이터 제거

In [74]:
import pandas as pd

df = pd.DataFrame({
    'c1': ['a','a','b','a','b'],
    'c2': [1, 1, 1, 2, 2],
    'c3': [1, 1, 2, 2, 2]
})

In [75]:
# drop_duplicates() 메소드
delete_dup = df.drop_duplicates()
print(df)
print('\n')
print(delete_dup)

c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


In [76]:
# drop_duplicates() 메소드를 통해 특정 열의 중복 확인 후 제거
delete_col_dup = df.drop_duplicates(subset=['c2'])
print(df)
print('\n')
print(delete_col_dup)

c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


  c1  c2  c3
0  a   1   1
3  a   2   2


c2 열을 기준으로 중복을 판단할 경우, 0 행과 1,2 행이 중복되고  
3 행과 4 행이 중복되므로 0 행과 3 행 이외에 모두 제거한다.

### 3-1 단위 환산

In [80]:
import pandas as pd

df = pd.read_csv('../../data/part5/auto-mpg.csv', header=None)
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'name']

In [83]:
mpg_to_kpl = 1.60934/3.78541

df['kpl'] = df['mpg'] * mpg_to_kpl
df['kpl'] = df['kpl'].round(2)

df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.65
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.38
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.65
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,6.80
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,7.23


같은 데이터셋 내에서 다른 단위를 사용하거나, 필요한 단위가 아닐 경우  
데이터의 일관성 측면에서 문제가 발생 할 수 있다.  
따라서 해당 데이터 단위를 통일 시켜준다.

### 3-2 자료형 변환

In [92]:
import pandas as pd

df = pd.read_csv('../../data/part5/auto-mpg.csv', header=None)
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'name']

In [93]:
# 각 열의 데이터타입 확인
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model year        int64
origin            int64
name             object
dtype: object

1. horsepower 는 int 가 적절하다.
2. model year 와 origin 은 범주형이 적절하다.

In [86]:
# horsepower 열 고유값 출력
print(df['horsepower'].unique())

['130.0' '165.0' '150.0' '140.0' '198.0' '220.0' '215.0' '225.0' '190.0'
 '170.0' '160.0' '95.00' '97.00' '85.00' '88.00' '46.00' '87.00' '90.00'
 '113.0' '200.0' '210.0' '193.0' '?' '100.0' '105.0' '175.0' '153.0'
 '180.0' '110.0' '72.00' '86.00' '70.00' '76.00' '65.00' '69.00' '60.00'
 '80.00' '54.00' '208.0' '155.0' '112.0' '92.00' '145.0' '137.0' '158.0'
 '167.0' '94.00' '107.0' '230.0' '49.00' '75.00' '91.00' '122.0' '67.00'
 '83.00' '78.00' '52.00' '61.00' '93.00' '148.0' '129.0' '96.00' '71.00'
 '98.00' '115.0' '53.00' '81.00' '79.00' '120.0' '152.0' '102.0' '108.0'
 '68.00' '58.00' '149.0' '89.00' '63.00' '48.00' '66.00' '139.0' '103.0'
 '125.0' '133.0' '138.0' '135.0' '142.0' '77.00' '62.00' '132.0' '84.00'
 '64.00' '74.00' '116.0' '82.00']


변환과정에서 누락데이터 ? 에 의해 전체 열이 문자열로 인식되었음을 알 수 있따.  
따라서 해당 결측값을 NaN 으로 변환하고 전체를 float 형으로 변환하는 것이 옳다.

In [94]:
# 누락데이터('?')를 nan 으로 변경 후 nan 제거
import numpy as np

df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True)

df['horsepower'] = df['horsepower'].astype('float')

df['horsepower'].dtypes

dtype('float64')

1. 누락데이터 ? 를 numpy.nan 으로 변경
2. 누락데이터 포함 행 제거
3. 실수 변환 가능한 열의 값들을 실수로 변환

In [101]:
# 정수형 데이터를 범주형 데이터로 변경
print('before', df['origin'].unique())

df['origin'].replace({
    1: 'USA',
    2: 'EU',
    3: 'JPN'
}, inplace=True)

print('after', df['origin'].unique())
print(df['origin'].dtypes)
print('\n')

df['origin'] = df['origin'].astype('category')
print(df['origin'].unique())
print(df['origin'].dtypes)

before ['USA', 'JPN', 'EU']
Categories (3, object): ['USA', 'JPN', 'EU']
after ['USA', 'JPN', 'EU']
Categories (3, object): ['USA', 'JPN', 'EU']
category


['USA', 'JPN', 'EU']
Categories (3, object): ['USA', 'JPN', 'EU']
category


origin 열의 값이 숫자에서 object 자료형으로 변경된 것을 알 수 있다.  
이후 astype('category') 를 통해 해당 열의 전체 자료형을 category 자료형으로 변경 하였다.  

In [108]:
# model year 자료형 변경
print('before' ,df['model year'].dtypes)
print(df['model year'].unique())
print('\n')

df['model year'] = df['model year'].astype('category')
print('after' ,df['model year'].dtypes)
print(df['model year'].unique())

before category
[70, 71, 72, 73, 74, ..., 78, 79, 80, 81, 82]
Length: 13
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]


after category
[70, 71, 72, 73, 74, ..., 78, 79, 80, 81, 82]
Length: 13
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]


연도는 숫자로 해도 별 중요한 부분은 없지만, 데이터셋의 크기가 클 수록  
연도 처럼 딱히 숫자 자체의 크기가 의미가 없는 데이터의 경우 범주형으로 표현해  
고유의 기능은 유지하고 필요없는 부분을 제거하는 것이 옳다.

### 4-1 구간 분할

In [110]:
import pandas as pd
import numpy as np

df = pd.read_csv('../../data/part5/auto-mpg.csv', header=None)
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'name']

In [114]:
# 누락 데이터 제거 및 자료형 변경
df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True)
df['horsepower'] = df['horsepower'].astype('float')

데이터분석 알고리즘에서 연속 데이터를 그대로 사용하는것 보다는,  
연속된 데이터를 의미있는 구간으로 쪼개고 해당 나눠진 구간을 분석하는 것이 의미가 더 큰 경우가 많다.  
가격, 효율, 비용 등의 연속적인 데이터를 특정 구간으로 분할한 이산 데이터를 생성한다.  
  
연속 변수를 일정한 구간으로 나누고 각 구간을 범주형 이산 변수로 변환하는 과정을 ```구간 분할 (binning)``` 이라고 한다.

In [115]:
# np.historgram 메소드로 3개의 bin 으로 구분할 경계값의 리스트 계산
count, bin_dividers = np.histogram(df['horsepower'], bins=3)
print(bin_dividers)

[ 46.         107.33333333 168.66666667 230.        ]


위의 코드에서 bins=3 을 통해 46~107 ... 총 3개의 구간 리스트를 생성했음을 알 수 있다.  
3개의 구간을 위해 경계값이 총 4개가 필요하다.

In [118]:
# 3개의 bin 에 이름 지정
bin_names = ['저출력', '보통출력', '고출력']

# pd.cut() 메소드를 통해 열의 데이터를 3개의 bin 에 할당
df['horsepower_bin'] = pd.cut(
                            x=df['horsepower'],
                            bins=bin_dividers,
                            labels=bin_names,
                            include_lowest=True
                            )

print(df[['horsepower', 'horsepower_bin']].head(10))

horsepower horsepower_bin
0       130.0           보통출력
1       165.0           보통출력
2       150.0           보통출력
3       150.0           보통출력
4       140.0           보통출력
5       198.0            고출력
6       220.0            고출력
7       215.0            고출력
8       225.0            고출력
9       190.0            고출력


pd.cut 메소드의 인자는,  
1. x = 분할 할 시리즈 데이터
2. bins = 분할 할 경계값 리스트
3. labels = 각 bin 의 이름
4. 각 구간의 첫 경계값 포함 여부

### 4-2 더미 변수

In [121]:
import pandas as pd
import numpy as np

df = pd.read_csv('../../data/part5/auto-mpg.csv', header=None)
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'name']

# 누락 데이터 제거 및 자료형 변경
df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True)
df['horsepower'] = df['horsepower'].astype('float')

# np.histogram 을 통해 3개의 bin 에 대한 경계값 리스트 계산
count, bin_dividers = np.histogram(df['horsepower'], bins=3)

# 3개 bin 이름 지정
bin_names = ['저출력', '보통출력' ,'고출력']

# pd.cut 을 통해 새로운 범주형 자료형 열 생성
df['hp_bin'] = pd.cut(
    x=df['horsepower'],
    bins=bin_dividers,
    labels=bin_names,
    include_lowest=True
)

In [124]:
# hp_bin 열의 범주형 데이터를 더미 변수로 변환
horsepower_dummies = pd.get_dummies(df['hp_bin'])

print(horsepower_dummies.head(15))
print('\n')
print(type(horsepower_dummies))

저출력  보통출력  고출력
0     0     1    0
1     0     1    0
2     0     1    0
3     0     1    0
4     0     1    0
5     0     0    1
6     0     0    1
7     0     0    1
8     0     0    1
9     0     0    1
10    0     0    1
11    0     1    0
12    0     1    0
13    0     0    1
14    1     0    0


<class 'pandas.core.frame.DataFrame'>


이전의 코드에서 연속된 값을 가지는 데이터를 범주형 데이터로 변경하였다.  
하지만, 실제 머신러닝 회귀분석등에서 사용할 때는 바로 사용 할 수 없는 경우가 잦고,  
이를 위해 범주형 데이터를 컴퓨터가 인식가능한 입력값으로 변환해야 한다.  
  
이럴떄 숫자 0 또는 1 로 표현되는 ```더미변수 (dummy variable)``` 를 사용한다.  
여기서 0 또는 1은 해당 특성이 존재하는지 없는지만을 의미한다.  
  
범주형 데이터를 컴퓨터가 인식 할 수 있도록 0, 1 로 구성되는 ```원핫벡터 (one hot vector)``` 로 바꾸는 것을  
```원핫인코딩 (one-hot-encoding)``` 이라고 한다.

In [134]:
# sklearn 라이브러리를 통한 원핫인코딩
from sklearn import preprocessing

# 전처리를 위한 encoder 객체 생성
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

# (1) label encoder 를 통해 문자열 범주를 숫자형 범주로 변환
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))
print(onehot_labeled, len(onehot_labeled))
print(type(onehot_labeled))
print('\n')

# (2) 2차원 행렬로 변경
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1)
print(onehot_reshaped)
print(type(onehot_reshaped))
print('\n')

# (3) 희소행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))


[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2] 15
<class 'numpy.ndarray'>


[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]]
<class 'numpy.ndarray'>


  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
<class 'scipy.sparse.csr.csr_matrix'>


1. preprocessing.LabelEncoder() 를 통해 문자열 범주를 숫자형 범주로 변환
2. reshapre() 메소드를 통해 2차원 행렬로 변경
3. preprocessing.OneHotEncoder() 를 통해 2차원 행렬을 희소행렬로 변경
  
1번 과정에 의해 저출력, 보통출력, 고출력은 각각 0,1,2 의 숫자형 범주로 변환되고  
2번 과정에 의해 각 행에 대해 숫자형 범주를 적용한 값을 2차원 배열로 변환한다.  
마지막으로 3번 과정에 의해 변환된 2차원 배열을 희소행렬로 변환한다.


### 5-1 정규화

In [135]:
import pandas as pd
import numpy as np

df = pd.read_csv('../../data/part5/auto-mpg.csv', header=None)
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'name']

# 누락 데이터 제거 및 자료형 변경
df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True)
df['horsepower'] = df['horsepower'].astype('float')

In [136]:
# horsepower 열의 산술적 요약 정보 확인
df['horsepower'].describe()

count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64

각 변수(데이터프레임의 열)에 들어있는 숫자 데이터의 크기가 클 경우  
머신러닝 분석의 결과치가 달라질 수 있다.  
따라서 상대적인 숫자 데이터의 크기 차이를 제거하기 위해 각 열에 속하는 데이터 값을  
동일한 크기 기준으로 나눈 비율로 나타내는것을 ```정규화 (normalization)``` 이라고 한다.  

In [137]:
# 해당 열의 최대값으로 나누기
df['hp_div_max'] = df['horsepower'] / abs(df['horsepower'].max())
df['hp_div_max'].head(10)

0    0.565217
1    0.717391
2    0.652174
3    0.652174
4    0.608696
5    0.860870
6    0.956522
7    0.934783
8    0.978261
9    0.826087
Name: hp_div_max, dtype: float64

In [138]:
# 해당 열에서 최소값을 뺀  값을 분자로하고 최대값과 최소값의 차이를 분모로 하는 정규화
x_minus_min = df['horsepower'] - df['horsepower'].min()
max_minus_min = df['horsepower'].max() - df['horsepower'].min()
df['hp_max_min_nor'] = x_minus_min / max_minus_min

df['hp_max_min_nor'].head(10)

0    0.456522
1    0.646739
2    0.565217
3    0.565217
4    0.510870
5    0.826087
6    0.945652
7    0.918478
8    0.972826
9    0.782609
Name: hp_max_min_nor, dtype: float64

### 6-1 다른 자료형을 시계열 객체로 변환

In [139]:
import pandas as pd
df = pd.read_csv('../../data/part5/stock-data.csv')

print(df.head())
print('\n')
print(df.info())

Date  Close  Start   High    Low  Volume
0  2018-07-02  10100  10850  10900  10000  137977
1  2018-06-29  10700  10550  10900   9990  170253
2  2018-06-28  10400  10900  10950  10150  155769
3  2018-06-27  10900  10800  11050  10500  133548
4  2018-06-26  10800  10900  11000  10700   63039


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB
None


실제 데이터는 시간 데이터임에도 불구하고 파이썬에서 사용하는 시계열 자료형 datetime 이 아닌 문자열, 숫자 등 다양한  
자료형으로 존재 하는 경우가 많다.  
이를 파이썬에 맞추어 ```pd.to_datetime()``` 메소드를 통해 datetime 자료형으로 변환한다.  

In [140]:
# pd.to_datetime() 메소드
df['new_Date'] = pd.to_datetime(df['Date'])

print(df.head())
print('\n')
print(df.info())

Date  Close  Start   High    Low  Volume   new_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      20 non-null     object        
 1   Close     20 non-null     int64         
 2   Start     20 non-null     int64         
 3   High      20 non-null     int64         
 4   Low       20 non-null     int64         
 5   Volume    20 non-null     int64         
 6   new_Date  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.2+ KB
None


In [141]:
# 시계열 값으로 변환된 열을 새로운 행 인덱스로 지정
df.set_index('new_Date', inplace=True)

# 기존 Date 열은 삭제
df.drop('Date', axis=1, inplace=True)

print(df.head())
print('\n')
print(df.info())

Close  Start   High    Low  Volume
new_Date                                      
2018-07-02  10100  10850  10900  10000  137977
2018-06-29  10700  10550  10900   9990  170253
2018-06-28  10400  10900  10950  10150  155769
2018-06-27  10900  10800  11050  10500  133548
2018-06-26  10800  10900  11000  10700   63039


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20 entries, 2018-07-02 to 2018-06-01
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Close   20 non-null     int64
 1   Start   20 non-null     int64
 2   High    20 non-null     int64
 3   Low     20 non-null     int64
 4   Volume  20 non-null     int64
dtypes: int64(5)
memory usage: 960.0 bytes
None


### 6-2 Timestamp 를 Period 로 변환

In [147]:
import pandas as pd

dates = ['2021-01-02', '2021-02-01', '2021-02-12']

# 문자열 날짜 배열을 timestamp 배열로 변환
ts_dates = pd.to_datetime(dates)
print(ts_dates)
print('\n')

# timestamp 를 period 로 변환
pr_day = ts_dates.to_period(freq='D')
print(pr_day)
pr_mon = ts_dates.to_period(freq='M')
print(pr_mon)
pr_year = ts_dates.to_period(freq='Y')
print(pr_year)

DatetimeIndex(['2021-01-02', '2021-02-01', '2021-02-12'], dtype='datetime64[ns]', freq=None)


PeriodIndex(['2021-01-02', '2021-02-01', '2021-02-12'], dtype='period[D]', freq='D')
PeriodIndex(['2021-01', '2021-02', '2021-02'], dtype='period[M]', freq='M')
PeriodIndex(['2021', '2021', '2021'], dtype='period[A-DEC]', freq='A-DEC')


### 6-3 시계열 데이터 만들기

In [148]:
import pandas as pd

# Timestamp 배열 생성 - 월 간격, 월의 시작일 기준
ts_ms = pd.date_range(
    start='2020-01-01',
    end=None,
    periods=6,
    freq='MS',
    tz='Asia/Seoul'
)

print(ts_ms)

DatetimeIndex(['2020-01-01 00:00:00+09:00', '2020-02-01 00:00:00+09:00',
               '2020-03-01 00:00:00+09:00', '2020-04-01 00:00:00+09:00',
               '2020-05-01 00:00:00+09:00', '2020-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


In [150]:
# Timestamp 배열 생성 - 월 간격, 월의 마지막일 기준
ts_me = pd.date_range(
    start='2020-01-01',
    end=None,
    periods=6,
    freq='M',
    tz='Asia/Seoul'
)

print(ts_me)
print('\n')

# Timestamp 배열 생성 - 3개월(1분기) 간격, 월의 마지막일 기준
ts_3m = pd.date_range(
    start='2020-01-01',
    end=None,
    periods=6,
    freq='3M',
    tz='Asia/Seoul'
)

print(ts_3m)
print('\n')


DatetimeIndex(['2020-01-31 00:00:00+09:00', '2020-02-29 00:00:00+09:00',
               '2020-03-31 00:00:00+09:00', '2020-04-30 00:00:00+09:00',
               '2020-05-31 00:00:00+09:00', '2020-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')


DatetimeIndex(['2020-01-31 00:00:00+09:00', '2020-04-30 00:00:00+09:00',
               '2020-07-31 00:00:00+09:00', '2020-10-31 00:00:00+09:00',
               '2021-01-31 00:00:00+09:00', '2021-04-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3M')




### 6-4 시계열 데이터 활용

In [182]:
import pandas as pd

df = pd.read_csv('../../data/part5/stock-data.csv')

# 문자열인 날짜 데이터를 Timestamp 자료형으로 변경
df['new_Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,Start,High,Low,Volume,new_Date
0,2018-07-02,10100,10850,10900,10000,137977,2018-07-02
1,2018-06-29,10700,10550,10900,9990,170253,2018-06-29
2,2018-06-28,10400,10900,10950,10150,155769,2018-06-28
3,2018-06-27,10900,10800,11050,10500,133548,2018-06-27
4,2018-06-26,10800,10900,11000,10700,63039,2018-06-26


In [178]:
# dt 속성을 이용하여 timestamp 의 연/월/일 정보를 년/월/일 컬럼에 삽입
df['year'] = df['new_Date'].dt.year
df['month'] = df['new_Date'].dt.month
df['day'] = df['new_Date'].dt.day
df.head()

,Date,Close,Start,High,Low,Volume,new_Date,year,month,day
0,2018-07-02,10100,10850,10900,10000,137977,2018-07-02,2018,7,2
1,2018-06-29,10700,10550,10900,9990,170253,2018-06-29,2018,6,29
2,2018-06-28,10400,10900,10950,10150,155769,2018-06-28,2018,6,28
3,2018-06-27,10900,10800,11050,10500,133548,2018-06-27,2018,6,27
4,2018-06-26,10800,10900,11000,10700,63039,2018-06-26,2018,6,26


In [170]:
# dt 를 통해 추출한 값을 pd.to_period() 메소드를 사용해 period 로 변환
df['p_year'] = df['new_Date'].dt.to_period(freq='A')
df['p_month'] = df['new_Date'].dt.to_period(freq='M')
df.head()

,Date,Close,Start,High,Low,Volume,new_Date,year,month,day,p_year,p_month
0,2018-07-02,10100,10850,10900,10000,137977,2018-07-02,2018,7,2,2018,2018-07
1,2018-06-29,10700,10550,10900,9990,170253,2018-06-29,2018,6,29,2018,2018-06
2,2018-06-28,10400,10900,10950,10150,155769,2018-06-28,2018,6,28,2018,2018-06
3,2018-06-27,10900,10800,11050,10500,133548,2018-06-27,2018,6,27,2018,2018-06
4,2018-06-26,10800,10900,11000,10700,63039,2018-06-26,2018,6,26,2018,2018-06


In [183]:
# 날짜 인덱스 활용하기
df.set_index('new_Date', inplace=True)
print(df.head())
print('\n')
print(df.index)

Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977
2018-06-29  2018-06-29  10700  10550  10900   9990  170253
2018-06-28  2018-06-28  10400  10900  10950  10150  155769
2018-06-27  2018-06-27  10900  10800  11050  10500  133548
2018-06-26  2018-06-26  10800  10900  11000  10700   63039


DatetimeIndex(['2018-07-02', '2018-06-29', '2018-06-28', '2018-06-27',
               '2018-06-26', '2018-06-25', '2018-06-22', '2018-06-21',
               '2018-06-20', '2018-06-19', '2018-06-18', '2018-06-15',
               '2018-06-14', '2018-06-12', '2018-06-11', '2018-06-08',
               '2018-06-07', '2018-06-05', '2018-06-04', '2018-06-01'],
              dtype='datetime64[ns]', name='new_Date', freq=None)


Timestamp 로 구성된 열을 행 인덱스로 지정할 경우 DatetimeIndex 라는 고유 속성으로 변환된다.  
마찬가지로 Period 로 구성된 열을 행 인덱스로 지정할 경우 PeriodIndex 라는 고유 속성으로 변환된다.  
이와 같이 날짜 인덱스를 활용하면 시계열 데이터에 대한 인덱싱과 슬라이싱이 매우 편해진다.

In [187]:
# 날짜 인덱스를 통해 데이터 선택하기
df = df.sort_index() # AssertionError: <class 'numpy.ndarray'> 에러 발생시 정렬후 수행 할 것.

df_y = df['2018']
print(df_y)
print('\n')

df_m = df['2018-07']
print(df_m)
print('\n')




Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-06-01  2018-06-01  11900  11800  12100  11750   32062
2018-06-04  2018-06-04  11900  11900  12200  11700   25171
2018-06-05  2018-06-05  12150  11800  12250  11800   42485
2018-06-07  2018-06-07  11950  12200  12300  11900   49088
2018-06-08  2018-06-08  11950  11950  12200  11800   59258
2018-06-11  2018-06-11  11950  12000  12250  11950   62293
2018-06-12  2018-06-12  13200  12200  13300  12050  558148
2018-06-14  2018-06-14  13450  13200  13700  13150  347451
2018-06-15  2018-06-15  13400  13600  13600  12900  201376
2018-06-18  2018-06-18  12000  13400  13400  12000  309787
2018-06-19  2018-06-19  11300  11850  11950  11300  180656
2018-06-20  2018-06-20  11550  11200  11600  10900  308596
2018-06-21  2018-06-21  11200  11350  11750  11200  133002
2018-06-22  2018-06-22  11300  11250  11450  10750  134805
2018-06-25  2018-06-25  11150  11400  11450  11000   55519
2018-06-26  201